In [1]:
import polars as pl
from pprint import pprint

## Basics

In [30]:
vs_q = pl.scan_parquet('data/VS_Q1_2025.parquet')


In [37]:
q1 = vs_q.filter(
    pl.col('Region') != 'IMPORTED/GCC',
)

q2 = q1.select(
    unique_makes = pl.col('Make').cast(pl.Categorical).unique()
)

q3 = q1.select(
    InquiryDate = pl.col('InquiryDate').str.to_datetime('%Y-%m-%d %H:%M:%S').dt.date()
)

# q2 = q1.group_by(
#     pl.col('Make'),
# ).agg(
#     pl.col('Model').n_unique()
# )

In [8]:
q1.explain()

'Parquet SCAN [data/VS_Q1_2025.parquet]\nPROJECT */13 COLUMNS\nSELECTION: [(col("Region")) != ("IMPORTED/GCC")]'

In [38]:
pprint(q2.explain())


('SELECT [col("Make").strict_cast(Categorical(None, '
 'Physical)).unique().alias("unique_makes")]\n'
 'FROM\n'
 '  Parquet SCAN [data/VS_Q1_2025.parquet]\n'
 '  PROJECT 2/13 COLUMNS\n'
 '  SELECTION: [(col("Region")) != ("IMPORTED/GCC")]')


In [39]:
unique_makes = q2.collect()


In [40]:
dates = q3.collect()


In [42]:
unique_makes['unique_makes'].to_list()

['NISSAN',
 'MERCEDES BENZ',
 'TOYOTA',
 'GMC',
 'FORD',
 'PORSCHE',
 'HONDA',
 'TESLA',
 'MITSUBISHI',
 'VOLKSWAGEN',
 'RENAULT',
 'HYUNDAI',
 'KIA',
 'MG',
 'INFINITI',
 'LEXUS',
 'DODGE',
 'CHEVROLET',
 'JAC',
 'CADILLAC',
 'AUDI',
 'BENTLEY',
 'MAZDA',
 'LAND ROVER',
 'FIAT',
 'SUZUKI',
 'JEEP',
 'SUBARU',
 'SKODA',
 'BYD',
 'ISUZU',
 'CHANGAN',
 'TATA',
 'BMW',
 'JETOUR',
 'GREAT WALL',
 'PEUGEOT',
 'MAXUS',
 'MINI',
 'HUMMER',
 'LAMBORGHINI',
 'CHERY',
 'ASHOK LEYLAND',
 'HONGQI',
 'EXEED',
 'ALFA ROMEO',
 'LINCOLN',
 'VOLVO',
 'ROLLS ROYCE',
 'SMART',
 'GEELY',
 'JAGUAR',
 'CHRYSLER',
 'MCLAREN',
 'BESTUNE',
 'HAVAL',
 'FERRARI',
 'MASERATI',
 'XPENG',
 'ROX',
 'DAIHATSU',
 'DAEWOO',
 'GAC',
 'BAIC',
 'HERO',
 'ASTON MARTIN',
 'SATURN',
 'OPEL',
 'VICTORY',
 'MAHINDRA',
 'HINO',
 'DONGFENG',
 'DFSK',
 'BAJAJ',
 'KING LONG',
 'MERCURY',
 'JINBEI',
 'CMC',
 'CITROEN',
 'JMC',
 'BUGATTI',
 'SSANGYONG',
 'POLESTAR',
 'YUTONG',
 'HARLEY DAVIDSON',
 'AVATR',
 'ABARTH',
 'SKYWELL ',
 '

In [67]:
q4 = q1.filter(
    pl.col('Make').is_not_null(),
    pl.col('Model').is_not_null(),
).group_by(
    pl.col('Make'),
).agg(
    pl.col('Model').unique()
)

In [50]:
pprint(q4.explain())

('AGGREGATE\n'
 '  [col("Model").unique()] BY [col("Make")]\n'
 '  FROM\n'
 '  Parquet SCAN [data/VS_Q1_2025.parquet]\n'
 '  PROJECT 3/13 COLUMNS\n'
 '  SELECTION: [([(col("Model").is_not_null()) & ([(col("Region")) != '
 '("IMPORTED/GCC")])]) & (col("Make").is_not_null())]')


In [73]:
make_models = q4.collect()
makes = make_models['Make'].to_list()
models = make_models['Model'].to_list()
make_model_dict = {make: models for make, models in zip(makes, models)}
make_model_dict


{'BENTLEY': ['MULSANNE',
  'AZURE',
  'BROOKLANDS',
  'FLYING SPUR',
  'BENTAYGA',
  'CONTINENTAL',
  'ARNAGE'],
 'HIPHI': ['Z'],
 'HONGQI': ['H7',
  'OUSADO',
  'H9',
  'E-HS9',
  'H6',
  'HS7',
  'E-QM5',
  '',
  'H5',
  'HS3',
  'HS5'],
 'JAC': ['S3',
  'S4',
  'J4',
  '',
  'E30X',
  'S3 PLUS',
  'J5',
  'S7',
  'M4',
  'SUNRAY',
  'IEV6',
  'M1',
  'JS4',
  'M5',
  'J7',
  'JS6',
  'T8',
  'J6'],
 'LADA': ['NIVA'],
 'POLESTAR': ['2', '4'],
 'NETA': ['NETA U'],
 'DONGFENG': ['T5',
  'AX7',
  'FORTHING',
  'VENUCIA',
  'AEOLUS A30',
  'SHINE',
  'M-HERO',
  'JINGYI X5',
  'K33-513',
  'MAGE'],
 'FERRARI': ['812',
  'F12',
  'PORTOFINO',
  '458',
  'LAFERRARI',
  'SF90',
  'SF90 STRADALE',
  '488',
  'F8 SPIDER',
  '612',
  'F8',
  'F8 TRIBUTO',
  '360',
  '599',
  'SF90 SPIDER',
  'F430',
  '296',
  'ROMA',
  'ENZO',
  'CALIFORNIA',
  'PUROSANGUE',
  'DAYTONA',
  'GTC4',
  'FF',
  'F355'],
 'SAMSUNG': ['SM3'],
 'MINI': ['COOPER',
  '',
  'COUPE',
  'COUNTRYMAN',
  'JOHN COOPER WORKS

In [185]:
make_count_q = q1.filter(
    pl.col('Make').is_not_null(),
).select(
    pl.col('Make').cast(pl.Categorical)
).group_by(
    pl.col('Make'),
).agg(
    pl.col('Make').count().alias('make_count'),
)
make_count_q = make_count_q.with_columns(
    total_queries = pl.col('make_count').sum().alias('total_queries')
)

make_count_q = make_count_q.select(
    make = pl.col('Make'),
    make_count_percentage = (pl.col('make_count') / pl.col('total_queries') * 100).cast(pl.Float32)
)

make_count = make_count_q.collect()
make_count.sort(by='make_count_percentage', descending=True).top_k(10, by='make_count_percentage').to_pandas()


,make,make_count_percentage
0,TOYOTA,20.847767
1,NISSAN,16.581478
2,HYUNDAI,5.936525
3,LEXUS,5.377803
4,MERCEDES BENZ,5.340772
5,MITSUBISHI,4.166034
6,HONDA,3.670574
7,KIA,3.578052
8,FORD,3.380593
9,CHEVROLET,3.327559


In [70]:
vv_q = pl.scan_parquet('data/VV_Q1_2025.parquet')


In [113]:
price_bracket_q = vv_q.filter(
    pl.col('Price').is_not_null(),
).select(
    pl.col('Price'),
    pl.col('Price').cut(
        breaks=[50000, 100000, 250000, 500000, 1000000],
        labels=['<50K', '50K to 100K', '100K to 250K', '250K to 500K', '500K to 1M', '>1M'],
    ).alias('PriceBracket'),
)

In [114]:
pprint(price_bracket_q.explain())

('SELECT [col("Price"), col("Price").cut().alias("PriceBracket")]\n'
 'FROM\n'
 '  Parquet SCAN [data/VV_Q1_2025.parquet]\n'
 '  PROJECT 1/33 COLUMNS\n'
 '  SELECTION: col("Price").is_not_null()')


In [116]:
price_bracket = price_bracket_q.collect()



In [117]:
price_bracket

Price,PriceBracket
f64,cat
710477.49,"""500K to 1M"""
29102.0,"""<50K"""
29102.0,"""<50K"""
70390.0,"""50K to 100K"""
60770.02,"""50K to 100K"""
…,…
104031.0,"""100K to 250K"""
59461.0,"""50K to 100K"""
77204.21,"""50K to 100K"""


## Tests

In [ ]:
makes = ['TOYOTA', 'NISSAN']
vs_q.with_columns(
    pl.col('Make').cast(pl.Categorical)

).filter(
    pl.col('Make').is_in(makes)
).collect()

VIN,Make,Model,CompanyName,ModelYear,Trim,EngineSize,Transmission,InquiryDate,Region,Colour,FuelType,BodyType
str,cat,str,str,str,str,str,str,str,str,str,str,str
"""JN8AY2NY3G9218676""","""NISSAN""","""PATROL""","""Methaq Takaful Insurance Compa…","""2016""","""LE""","""5.6 L""","""AUTOMATIC""","""2025-03-05 21:57:55""","""GCC""",null,"""Petrol""","""SUV"""
"""JTEGD52M08A004048""","""TOYOTA""","""PREVIA""","""Methaq Takaful Insurance Compa…","""2008""","""DLX""","""2.4 L""","""AUTOMATIC""","""2025-03-05 21:57:56""","""GCC""",null,"""Petrol""","""MPV"""
"""6T1BE42K58X499283""","""TOYOTA""","""CAMRY""","""Methaq Takaful Insurance Compa…","""2008""","""GL""","""2.4 L""","""AUTOMATIC""","""2025-03-05 21:57:56""","""GCC""",null,"""Petrol""","""SEDAN"""
"""4T1B11HK9JU645348""","""TOYOTA""","""CAMRY""","""Oman Qatar Insurance Company""","""2018""","""LIMITED""","""2.5 L""","""AUTOMATIC""","""2025-03-05 21:57:56""","""NON-GCC""",null,"""Petrol""","""SEDAN"""
"""JN8AY2NYXK9367742""","""NISSAN""","""PATROL""","""United Fidelity Insurance Comp…","""2019""","""NISMO""","""5.6 L""","""AUTOMATIC""","""2025-03-05 21:57:57""","""GCC""",null,"""Petrol""","""SUV"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""MDHBN7ADXDG015932""","""NISSAN""","""SUNNY""","""Rasan""","""2013""","""SL""","""1.5 L""","""AUTOMATIC""","""2025-04-30 23:59:46""","""GCC""",null,"""Petrol""","""SEDAN"""
"""JN6DD23S29X242286""","""NISSAN""","""PICK UP""","""Rasan""","""2009""","""DX""","""2.4 L""","""MANUAL""","""2025-04-30 23:59:51""","""GCC""",null,"""Petrol""","""PICK UP SINGLE CAB"""
"""JTDBV42E0C9022901""","""TOYOTA""","""COROLLA""","""Rasan""","""2012""","""XLI""","""1.6 L""","""MANUAL""","""2025-04-30 23:59:52""","""GCC""",null,"""Petrol""","""SEDAN"""


## Pivoting


In [13]:
vsp = pl.scan_parquet('data/VS_Q1_2025_processed.parquet')


In [14]:
year_ranges_enum = pl.Enum(['1900 to 2000', '2000 to 2010', '2010 to 2015', '2015 to 2020', '2020 to 2024', '2025 to Present'])

In [15]:
top_makes = vsp.group_by(
    pl.col('Make'),
).agg(
    pl.col('Make').count().alias('make_count')
).sort(
    by='make_count', descending=True
).select(
    pl.col('Make')
).limit(10) 


In [8]:
q = (
    vsp.join(top_makes, on='Make', how='inner')\
    .select(
        pl.col('Make').cast(pl.Categorical),
        pl.col('ModelYearRanges').cast(year_ranges_enum),
        pl.col('InquiryDate')
    )
    .collect()\
    .pivot(
        index='Make', 
        on='ModelYearRanges', 
        values='InquiryDate', 
        aggregate_function='len')\
    .lazy()
)

out = q.collect().to_pandas()


In [16]:
out

,Make,1900 to 2000,2000 to 2010,2010 to 2015,2015 to 2020,2020 to 2024,2025 to Present
0,KIA,131,8024,44433,48275,36604,2299
1,HYUNDAI,247,19169,83939,101293,44592,4931
2,CHEVROLET,1085,15853,31977,40827,35047,3245
3,MITSUBISHI,1590,31304,37531,49889,25567,2221
4,HONDA,1592,40395,34663,36811,18829,577
5,MERCEDES BENZ,23472,11137,37476,67133,45734,3429
6,TOYOTA,20427,170531,220455,175429,174576,18247
7,FORD,733,16182,52681,40018,22418,2181
8,LEXUS,11188,54264,42462,43417,33327,2409
9,NISSAN,9355,82700,161847,199036,125130,10144


In [23]:
import polars as pl

grouped_and_counted_lazy = (
    vsp.join(top_makes, on='Make', how='inner')
    .select(
        pl.col('Make').cast(pl.Categorical),
        pl.col('ModelYearRanges').cast(year_ranges_enum),
    )
    .group_by(['Make', 'ModelYearRanges'])
    .agg(
        pl.len().alias('count') # pl.len() counts the number of rows in each group
    )
)

pivoted_result = (
    grouped_and_counted_lazy
    .collect()
    .sort(by='ModelYearRanges') # Collects the Make, ModelYearRanges, count table
    .pivot(
        index='Make',
        on='ModelYearRanges',
        values='count'
    )
    .lazy() # Make it lazy again if you have more steps
)

out = pivoted_result.collect().to_pandas()

In [28]:
out

,Make,1900 to 2000,2000 to 2010,2010 to 2015,2015 to 2020,2020 to 2024,2025 to Present
0,NISSAN,9355,82700,161847,199036,125130,10144
1,FORD,733,16182,52681,40018,22418,2181
2,MITSUBISHI,1590,31304,37531,49889,25567,2221
3,TOYOTA,20427,170531,220455,175429,174576,18247
4,MERCEDES BENZ,23472,11137,37476,67133,45734,3429
5,KIA,131,8024,44433,48275,36604,2299
6,CHEVROLET,1085,15853,31977,40827,35047,3245
7,HONDA,1592,40395,34663,36811,18829,577
8,HYUNDAI,247,19169,83939,101293,44592,4931
9,LEXUS,11188,54264,42462,43417,33327,2409


In [27]:
out.set_index('Make').fillna(0, inplace=True)

## Trends

In [31]:
q = (
    vs_q.select(
        pl.col('InquiryDate').str.to_datetime('%Y-%m-%d %H:%M:%S').dt.date()
    )
    .sort('InquiryDate')
    .group_by_dynamic(
        index_column='InquiryDate',
        every='1d'
    )
    .agg(
        pl.len().alias('daily_inquiries')
    )
)

In [33]:
q.collect()

InquiryDate,daily_inquiries
date,u32
2024-12-31,35100
2025-01-01,7305
2025-01-02,48051
2025-01-03,39210
2025-01-04,29956
…,…
2025-04-26,95990
2025-04-27,120639
2025-04-28,137948


## Claims

In [2]:
cmf_q = pl.scan_parquet('data/Claims.parquet')

In [77]:
full_specs_cmf = cmf_q.group_by(
    pl.col('Make')
).agg(
    pl.col('RepairValue').mean().alias('count')
).filter(
    pl.col('count').is_not_null()
).sort(
    by=pl.col('count'),
    descending=True
).collect()

In [78]:
full_specs_cmf.head(10)

Make,count
str,f64
"""FERRARI""",71795.766667
"""LAMBORGHINI""",70488.602679
"""ROLLS ROYCE""",32591.489796
"""BENTLEY""",28812.057333
"""ASTON MARTIN""",21100.0
"""MCLAREN""",18714.2
"""LAND ROVER""",12274.536553
"""TESLA""",10745.2011
"""MASERATI""",10261.079044


In [26]:
grouped_data = (
    cmf_q.select(
        pl.col('FullSpec').cast(
            pl.String).cast(pl.Categorical)
    )
    .group_by(
        pl.col('FullSpec')
    )
    .agg(
        pl.col('FullSpec').count().alias('count')
    )
).collect()

In [27]:
grouped_data

FullSpec,count
cat,u32
"""LAMBORGHINI HURACAN 2016""",1
"""ROLLS ROYCE GHOST 2013""",5
"""LAMBORGHINI URUS 2019""",1
"""JAGUAR XE 2017""",3
"""MERCEDES BENZ AMG 2017""",21
…,…
"""CADILLAC """,1
"""JAGUAR X TYPE """,1
""" ELANTRA """,1


In [37]:
grouped_data = grouped_data.with_columns(
    normalized_count=(100 * pl.col('count') / pl.col('count').sum()).cast(pl.Float32)
)

In [38]:
grouped_data = grouped_data.sort(
    by='normalized_count', descending=True
)

In [3]:
company_group = cmf_q.group_by(
    pl.col('CompanyName')
).agg(
    pl.len().alias('count')
).collect()

In [5]:
company_group.to_arrow()

pyarrow.Table
CompanyName: large_string
count: uint32
----
CompanyName: [["United Fidelity Insurance Company","At Fault","Adamjee","InsureFlow","Arabia"]]
count: [[13434,25621,7839,58617,3123]]

## Preprocessing

In [66]:
import json

with open('data/make_countries.json', 'r') as f:
    make_countries = json.load(f)


In [79]:
claims_pq = pl.scan_csv('data/Claims.csv', schema_overrides=
                        {
                            'Accident Report ID': pl.String
                        })

In [ ]:

vv_pq = vv_q.with_columns(
    pl.col('Price').cut(
        breaks=[50000, 100000, 250000, 500000, 1000000],
        labels=['<50K', '50K to 100K', '100K to 250K', '250K to 500K', '500K to 1M', '>1M'],
    ).alias('PriceBracket'),
    pl.when(pl.col('Make').is_in(list(make_countries.keys())))\
    .then(pl.col('Make').replace(make_countries))\
    .otherwise(pl.col('Make'))\
    .alias('Country')
)

vs_pq = vs_q.filter(
    pl.col('EngineSize').is_not_null(),
    pl.col('ModelYear').is_not_null(),
    pl.col('FuelType').is_not_null(),
    pl.col('Make').is_not_null(),
    pl.col('Model').is_not_null(),
    pl.col('InquiryDate').is_not_null(),
    pl.col('EngineSize') != ' ',
    pl.col('EngineSize') != '',
    
).with_columns(
    pl.col('EngineSize').str.replace('L', '').str.replace(' ', '').cast(pl.Float32).alias('EngineSizeNumeric'),
    pl.col('ModelYear').cast(pl.Int32).alias('ModelYearNumeric'),
    # isev
    pl.when(pl.col('FuelType').str.contains('Electricity') | pl.col('FuelType').str.contains('Petrol + Electricity') ).then(True).otherwise(False).alias('IsEV'),
)

vs_pq = vs_pq.with_columns(
    pl.col('ModelYearNumeric').cut(
        breaks=[2000, 2010, 2015, 2020, 2024],
        labels=['1900 to 2000', '2000 to 2010', '2010 to 2015', '2015 to 2020', '2020 to 2024', '2025 to Present'],
    ).alias('ModelYearRanges'),
    pl.col('EngineSizeNumeric').cut(
        breaks=[2.0, 3.0, 4.0, 5.0],
        labels=['<2.0L', '2.0 to 3.0L', '3.0 to 4.0L', '4.0 to 5.0L', '>5.0L'],
    ).alias('EngineSizeBracket'),
    pl.when(pl.col('Make').is_in(list(make_countries.keys())))\
    .then(pl.col('Make').replace(make_countries))\
    .otherwise(pl.col('Make'))\
    .alias('Country')
)


In [80]:

claims_pq = claims_pq.with_columns(
    pl.when(pl.col('Make').is_in(list(make_countries.keys())))\
    .then(pl.col('Make').replace(make_countries))\
    .otherwise(pl.col('Make'))\
    .alias('Country')
)

In [81]:
pprint(claims_pq.explain())

(' WITH_COLUMNS:\n'
 ' [when(col("Make").is_in([["MERCEDES BENZ", "BMW", … '
 '"LADA"]])).then(col("Make").replace([Series.implode(), '
 'Series.implode()])).otherwise(col("Make")).alias("Country")] \n'
 '  Csv SCAN [data/Claims.csv]\n'
 '  PROJECT */25 COLUMNS')


In [73]:
pprint(vv_pq.explain())

(' WITH_COLUMNS:\n'
 ' [col("Price").cut().alias("PriceBracket"), '
 'when(col("Make").is_in([["MERCEDES BENZ", "BMW", … '
 '"LADA"]])).then(col("Make").replace([Series.implode(), '
 'Series.implode()])).otherwise(col("Make")).alias("Country")] \n'
 '  Parquet SCAN [data/VV_Q1_2025.parquet]\n'
 '  PROJECT */33 COLUMNS')


In [74]:
pprint(vs_pq.explain())


('simple π 19/19 ["VIN", "Make", "Model", ... 16 other columns]\n'
 '   WITH_COLUMNS:\n'
 '   [col("ModelYearNumeric").cut().alias("ModelYearRanges"), '
 'col("EngineSizeNumeric").cut().alias("EngineSizeBracket")] \n'
 '     WITH_COLUMNS:\n'
 '     [col("EngineSize").str.replace(["L", ""]).str.replace([" ", '
 '""]).strict_cast(Float32).alias("EngineSizeNumeric"), '
 'col("ModelYear").strict_cast(Int32).alias("ModelYearNumeric"), '
 'when([(col("FuelType").str.contains(["Electricity"])) | '
 '(col("FuelType").str.contains(["Petrol + '
 'Electricity"]))]).then(true).otherwise(false).alias("IsEV"), '
 'when(col("Make").is_in([["MERCEDES BENZ", "BMW", … '
 '"LADA"]])).then(col("Make").replace([Series.implode(), '
 'Series.implode()])).otherwise(col("Make")).alias("Country")] \n'
 '      Parquet SCAN [data/VS_Q1_2025.parquet]\n'
 '      PROJECT */13 COLUMNS\n'
 '      SELECTION: [([([([([(col("InquiryDate").is_not_null()) & '
 '(col("FuelType").is_not_null())]) & (col("Make").is_not_null()

In [10]:
# vs_pq.collect().write_parquet('data/VS_Q1_2025_processed.parquet')
vv_pq.collect().write_parquet('data/VV_Q1_2025_processed.parquet')


NameError: name 'vv_pq' is not defined

In [76]:
vs_pq.collect().write_parquet('data/VS_Q1_2025_processed.parquet')


In [82]:
claims_pq.collect().write_parquet('data/Claims.parquet')